In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip list

# Libraries

In [3]:
import os
import json
import pandas as pd
import pandasql as ps

In [4]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [5]:
INPUT_PATH="input"
OUTPUT_PATH="output"
CONFIG_PATH="config"

INPUT_DATA="test_data2"
INPUT_DATA_CONFIG="input_config"
OUTPUT_DATA="output"

INPUT_EXTENSION="csv"
INPUT_CONFIG_EXTENSION="json"
OUTPUT_EXTENSION="csv"

INPUT_FILE=f"{INPUT_DATA}.{INPUT_EXTENSION}"
INPUT_CONFIG_FILE=f"{INPUT_DATA_CONFIG}.{INPUT_CONFIG_EXTENSION}"
OUTPUT_FILE=f"{OUTPUT_DATA}.{OUTPUT_EXTENSION}"

INPUT_ABS_APTH=os.path.abspath(os.path.join(INPUT_PATH, INPUT_FILE))
INPUT_FILE_CONFIG=os.path.abspath(os.path.join(CONFIG_PATH, INPUT_CONFIG_FILE))
OUTPUT_ABS_APTH=os.path.abspath(os.path.join(OUTPUT_PATH, OUTPUT_FILE))

In [6]:
with open(INPUT_FILE_CONFIG, encoding='utf-8') as f:
    CONFIG = json.load(f)

In [7]:
CONFIG

{'INPUTS': {'FILE_NAME': ['test_data2.csv'],
  'SEPARATOR': ',',
  'DECIMAL': None,
  'ENCODING': 'utf-8',
  'FLOAT_PRECISION': 'high',
  'INDEXES': ['period_end_date', 'translated_when'],
  'DATE_COLUMNS': ['period_end_date', 'translated_when'],
  'DTYPE': {'if_data_corrected': 'object',
   'prod_gr_id': 'object',
   'country_id_n': 'object',
   'delivery_type_id': 'object',
   'freq_id': 'object',
   'retailer_id': 'object',
   'brand_id': 'object',
   'predict_automatch': 'float',
   'class_acctual': 'float'},
  'CATEGORICAL_FEATURES': ['country_id_n',
   'prod_gr_id',
   'retailer_id',
   'brand_id',
   'delivery_type_id',
   'week_number'],
  'COLUMNS_WITH_NAN_VALUES': [None]},
 'MODEL': {'TARGET': 'class_acctual',
  'PREDICTION': 'predict_automatch',
  'DATETIME': 'translated_when'},
 'OUTPUTS': {'NAME': [None],
  'COLUMNS_TO_EXCLUDE': ['if_data_corrected', 'freq_id'],
  'BREAKING_POINT_DT': '2020-11-28 00:00:00+00:00'}}

# Load data

In [8]:
df = pd.read_csv(
    INPUT_ABS_APTH,
    sep=CONFIG['INPUTS']['SEPARATOR'],
    encoding=CONFIG['INPUTS']['ENCODING'],
    infer_datetime_format=True,
    parse_dates=CONFIG['INPUTS']['DATE_COLUMNS'],
    engine="c",
    low_memory=False,
    skipinitialspace=True,
    dtype=CONFIG['INPUTS']['DTYPE']
)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19697 entries, 0 to 19696
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   period_end_date    19640 non-null  datetime64[ns, UTC]
 1   translated_when    19697 non-null  datetime64[ns, UTC]
 2   if_data_corrected  19697 non-null  object             
 3   prod_gr_id         19697 non-null  object             
 4   country_id_n       18405 non-null  object             
 5   delivery_type_id   18362 non-null  object             
 6   freq_id            19697 non-null  object             
 7   retailer_id        19697 non-null  object             
 8   brand_id           19697 non-null  object             
 9   predict_automatch  19368 non-null  float64            
 10  class_acctual      19697 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(2), object(7)
memory usage: 1.7+ MB


In [10]:
df.sort_values(by=['translated_when'], inplace=True)

In [11]:
df['translated_when'].unique()

<DatetimeArray>
['2020-09-01 03:05:51+00:00', '2020-09-01 03:35:26+00:00',
 '2020-09-01 03:44:29+00:00', '2020-09-01 06:15:32+00:00',
 '2020-09-01 06:15:50+00:00', '2020-09-01 06:21:31+00:00',
 '2020-09-01 06:24:04+00:00', '2020-09-01 06:24:13+00:00',
 '2020-09-01 06:25:53+00:00', '2020-09-01 06:26:45+00:00',
 ...
 '2021-01-25 18:04:52+00:00', '2021-01-26 08:17:06+00:00',
 '2021-01-26 08:28:42+00:00', '2021-01-26 14:22:20+00:00',
 '2021-01-26 14:52:17+00:00', '2021-01-27 17:35:47+00:00',
 '2021-01-28 10:44:57+00:00', '2021-01-28 13:48:19+00:00',
 '2021-01-29 09:56:17+00:00', '2021-02-01 14:50:49+00:00']
Length: 15628, dtype: datetime64[ns, UTC]

In [12]:
df.head(10)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
1908,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,0,426,126,22866,1,174,280,1.0,1.0
1907,2020-08-30 00:00:00+00:00,2020-09-01 03:35:26+00:00,0,426,126,22866,2,93,96,1.0,1.0
4503,2020-08-30 00:00:00+00:00,2020-09-01 03:44:29+00:00,0,413,113,12731,2,95,182,1.0,1.0
8009,2020-09-01 00:00:00+00:00,2020-09-01 06:15:32+00:00,1,426,114,18614,1,173,279,1.0,1.0
5091,2020-09-01 00:00:00+00:00,2020-09-01 06:15:50+00:00,1,426,114,18614,1,34,234,1.0,1.0
8010,2020-09-01 00:00:00+00:00,2020-09-01 06:21:31+00:00,1,426,114,18614,2,93,96,1.0,1.0
9945,2020-09-01 00:00:00+00:00,2020-09-01 06:24:04+00:00,1,426,114,18614,1,117,243,1.0,1.0
9944,2020-09-01 00:00:00+00:00,2020-09-01 06:24:13+00:00,1,426,114,18614,2,11,33,1.0,1.0
10999,2020-09-01 00:00:00+00:00,2020-09-01 06:25:53+00:00,1,426,114,18614,1,26,169,0.0,0.0
9947,2020-09-01 00:00:00+00:00,2020-09-01 06:26:45+00:00,1,426,114,18614,1,33,379,0.0,0.0


In [13]:
df.describe(
    include='all',
    datetime_is_numeric=True
)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
count,19640,19697,19697,19697,18405,18362,19697,19697,19697,19368.000000,19697.000000
unique,NaN,NaN,2,3,34,914,2,52,199,NaN,NaN
top,NaN,NaN,0,426,121,31480,2,30,33,NaN,NaN
freq,NaN,NaN,17085,11844,4153,998,11934,1197,587,NaN,NaN
mean,2020-10-14 23:30:22.729124096+00:00,2020-10-22 06:01:21.525206784+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.636307,0.743210
min,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,2020-09-27 00:00:00+00:00,2020-09-29 13:45:29+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
50%,2020-10-18 00:00:00+00:00,2020-10-22 04:30:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
75%,2020-11-08 00:00:00+00:00,2020-11-13 11:57:53+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
max,2020-12-01 00:00:00+00:00,2021-02-01 14:50:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000


**An example of model's underperformance:**

country_id == '106'

./docs/images/monthly_stability/class_acctual/country_id_n/CLASS_106_monthly_stability_grouped.jpg

Observed:

![image info](./docs/images/monthly_stability/class_acctual/country_id_n/CLASS_106_monthly_stability_grouped.jpg)


./docs/images/monthly_stability/class_acctual/country_id_n/CLASS_106_monthly_stability_grouped.jpg

Predicted:

![image info](./docs/images/monthly_stability/class_acctual/country_id_n/CLASS_106_monthly_stability_grouped.jpg)

In [14]:
df_sub = df[['period_end_date', 'translated_when', 'prod_gr_id', 'country_id_n', 'brand_id', 'predict_automatch', 'class_acctual']]

In [15]:
pysqldf(
    """
    SELECT
        'PREDICTED' AS type,
        strftime('%Y-%m', period_end_date) AS month_year
        ,country_id_n
        ,predict_automatch
        --,class_acctual
        ,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year
        ,country_id_n
        ,predict_automatch
        --,class_acctual
    ;
    """
)

,type,month_year,country_id_n,predict_automatch,predict_automatch_sum
0,PREDICTED,2020-08,106,0.0,0.0
1,PREDICTED,2020-08,106,1.0,1.0
2,PREDICTED,2020-09,106,0.0,0.0
3,PREDICTED,2020-09,106,1.0,115.0
4,PREDICTED,2020-10,106,NaN,NaN
5,PREDICTED,2020-10,106,0.0,0.0
6,PREDICTED,2020-10,106,1.0,108.0
7,PREDICTED,2020-11,106,0.0,0.0
8,PREDICTED,2020-11,106,1.0,89.0


In [16]:
pysqldf(
    """
    SELECT
        'OBSERVED' AS type,
        strftime('%Y-%m', period_end_date) AS month_year
        ,country_id_n
        --,predict_automatch
        ,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year
        ,country_id_n
        --,predict_automatch
        ,class_acctual
    ;
    """
)

,type,month_year,country_id_n,class_acctual,class_acctual_sum
0,OBSERVED,2020-08,106,0.0,0.0
1,OBSERVED,2020-08,106,1.0,2.0
2,OBSERVED,2020-09,106,0.0,0.0
3,OBSERVED,2020-09,106,1.0,189.0
4,OBSERVED,2020-10,106,0.0,0.0
5,OBSERVED,2020-10,106,1.0,166.0
6,OBSERVED,2020-11,106,0.0,0.0
7,OBSERVED,2020-11,106,1.0,171.0


In [17]:
pysqldf(
    """
    SELECT
        'OBSERVED' AS type,
        strftime('%Y-%m', period_end_date) AS month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    --WHERE
        --country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    ;
    """
)

,type,month_year,class_acctual_sum
0,OBSERVED,None,47.0
1,OBSERVED,2020-08,240.0
2,OBSERVED,2020-09,4306.0
3,OBSERVED,2020-10,4675.0
4,OBSERVED,2020-11,5369.0
5,OBSERVED,2020-12,2.0


In [18]:
pysqldf(
    """
    SELECT
        'PREDICTED' AS type,
        strftime('%Y-%m', period_end_date) AS month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
        ,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    --WHERE
        --country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    ;
    """
)

,type,month_year,predict_automatch_sum
0,PREDICTED,None,40.0
1,PREDICTED,2020-08,209.0
2,PREDICTED,2020-09,3612.0
3,PREDICTED,2020-10,3974.0
4,PREDICTED,2020-11,4488.0
5,PREDICTED,2020-12,1.0


In [19]:
pysqldf(
    """
    SELECT
        'OBSERVED' AS type,
        strftime('%Y-%m', translated_when) AS month_year_translated
        --,country_id_n
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    --WHERE
        --country_id_n = '106'
        --AND month_year = '2020-11'
    GROUP BY
        month_year_translated
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    ;
    """
)

,type,month_year_translated,class_acctual_sum
0,OBSERVED,2020-09,3800.0
1,OBSERVED,2020-10,4823.0
2,OBSERVED,2020-11,5320.0
3,OBSERVED,2020-12,583.0
4,OBSERVED,2021-01,112.0
5,OBSERVED,2021-02,1.0


# Something fishy

# Demand or Supplie dropped ? A product got out of fashion?

In [20]:
pysqldf(
    """
    SELECT DISTINCT
        prod_gr_id
        ,COUNT(prod_gr_id)
    FROM (
    SELECT
        strftime('%Y-%m', period_end_date) AS month_year
        ,translated_when
        ,prod_gr_id
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        strftime('%Y', translated_when) < '2021'
        --country_id_n = '105'
        --AND month_year = '2020-11'
    --GROUP BY
        --month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    )
    GROUP BY
    prod_gr_id
    ;
    """
)

,prod_gr_id,COUNT(prod_gr_id)
0,413,4482
1,426,11701
2,427,3362


In [21]:
pysqldf(
    """
    SELECT DISTINCT
        prod_gr_id
        ,COUNT(prod_gr_id)
    FROM (
    SELECT
        strftime('%Y-%m', period_end_date) AS month_year
        ,translated_when
        ,prod_gr_id
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        strftime('%Y', translated_when) >= '2021'
        --country_id_n = '105'
        --AND month_year = '2020-11'
    --GROUP BY
        --month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    )
    GROUP BY
    prod_gr_id
    ;
    """
)

,prod_gr_id,COUNT(prod_gr_id)
0,413,4
1,426,143
2,427,5


#### Find brand ids from single prod_gr_id

In [22]:
prod_gr_id_x_brand_id = pysqldf(
    """
    SELECT DISTINCT
         prod_gr_id
        ,brand_id
        ,COUNT(brand_id) as brand_id_count
        --,country_id_n
        --,predict_automatch
        --,class_acctual
        --,SUM(predict_automatch) AS predict_automatch_sum
        --,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    --WHERE
        --country_id_n = '105'
        --AND month_year = '2020-11'
    GROUP BY
         brand_id
        ,prod_gr_id
        --month_year
        --,country_id_n
        --,predict_automatch
        --,class_acctual
    ORDER BY
        prod_gr_id
        ,brand_id ASC
        
    ;
    """
)

In [23]:
import pprint as pp

In [24]:
for gr in prod_gr_id_x_brand_id['prod_gr_id'].unique():
    print("\n", gr)
    print(prod_gr_id_x_brand_id[prod_gr_id_x_brand_id['prod_gr_id'] == gr].pivot(index='prod_gr_id', columns='brand_id', values='brand_id_count'))


 413
brand_id    116  147  178  179  180  181  182  183  184  185  ...  369   37  \
prod_gr_id                                                    ...             
413          98  115   38  126  240  201   94  129   15   38  ...    1  178   

brand_id    38  49   60  71  82  83   84  85  
prod_gr_id                                    
413         59  52  114  86  24  32  120  40  

[1 rows x 88 columns]

 426
brand_id    108  111  114  123  128  129  132  135  151  169  ...  58   60  \
prod_gr_id                                                    ...            
426         272  337   42   81   43  113  200   56   27  254  ...  78  114   

brand_id     65  74   75   76  77   95   96   99  
prod_gr_id                                        
426         334  99  247  285  86  359  579  527  

[1 rows x 88 columns]

 427
brand_id    108  111  114  123  127  130  132  135  147  150  ...  329  362  \
prod_gr_id                                                    ...             
427        

In [25]:
prod_gr_id_x_brand_id

,prod_gr_id,brand_id,brand_id_count
0,413,116,98
1,413,147,115
2,413,178,38
3,413,179,126
4,413,180,240
...,...,...,...
249,427,427,5
250,427,429,48
251,427,430,88
252,427,433,1


In [26]:
df2 = prod_gr_id_x_brand_id.pivot(index='brand_id', columns='prod_gr_id', values='brand_id_count').reset_index()
df2 

prod_gr_id,brand_id,413,426,427
0,108,NaN,272.0,48.0
1,111,NaN,337.0,32.0
2,114,NaN,42.0,56.0
3,116,98.0,NaN,NaN
4,123,NaN,81.0,82.0
...,...,...,...,...
194,85,40.0,NaN,NaN
195,90,NaN,NaN,38.0
196,95,NaN,359.0,NaN
197,96,NaN,579.0,NaN


In [27]:
df2[df2['413'] == df2['413'].max()]


prod_gr_id,brand_id,413,426,427
35,180,240.0,NaN,12.0


#### Large FN when product_gr_id 413 and brand id 180

In [28]:
prod_gr_id_x_brand_id_CM = pysqldf(
    """
    SELECT DISTINCT
         prod_gr_id
        ,brand_id
        ,COUNT(brand_id) as brand_id_count
        --,country_id_n
        ,predict_automatch
        ,class_acctual
        ,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        brand_id = '180'
        --AND month_year = '2020-11'
    GROUP BY
         brand_id
        ,prod_gr_id
        --month_year
        --,country_id_n
        ,predict_automatch
        ,class_acctual
    ORDER BY
        prod_gr_id
        ,brand_id ASC
        
    ;
    """
)

In [29]:
prod_gr_id_x_brand_id_CM

,prod_gr_id,brand_id,brand_id_count,predict_automatch,class_acctual,predict_automatch_sum,class_acctual_sum
0,413,180,48,0.0,0.0,0.0,0.0
1,413,180,40,0.0,1.0,0.0,40.0
2,413,180,12,1.0,0.0,12.0,0.0
3,413,180,140,1.0,1.0,140.0,140.0
4,427,180,3,0.0,0.0,0.0,0.0
5,427,180,1,1.0,0.0,1.0,0.0
6,427,180,8,1.0,1.0,8.0,8.0


#### Large FN when product_gr_id 426 and brand id 96

In [30]:
prod_gr_id_x_brand_id_CM = pysqldf(
    """
    SELECT DISTINCT
         prod_gr_id
        ,brand_id
        ,COUNT(brand_id) as brand_id_count
        --,country_id_n
        ,predict_automatch
        ,class_acctual
        ,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        brand_id = '96'
        --AND month_year = '2020-11'
    GROUP BY
         brand_id
        ,prod_gr_id
        --month_year
        --,country_id_n
        ,predict_automatch
        ,class_acctual
    ORDER BY
        prod_gr_id
        ,brand_id ASC
        
    ;
    """
)

In [31]:
prod_gr_id_x_brand_id_CM

,prod_gr_id,brand_id,brand_id_count,predict_automatch,class_acctual,predict_automatch_sum,class_acctual_sum
0,426,96,1,NaN,0.0,NaN,0.0
1,426,96,117,0.0,0.0,0.0,0.0
2,426,96,95,0.0,1.0,0.0,95.0
3,426,96,40,1.0,0.0,40.0,0.0
4,426,96,326,1.0,1.0,326.0,326.0


#### Large FN when product_gr_id 427 and brand id 166

In [32]:
prod_gr_id_x_brand_id_CM = pysqldf(
    """
    SELECT DISTINCT
         prod_gr_id
        ,brand_id
        ,COUNT(brand_id) as brand_id_count
        --,country_id_n
        ,predict_automatch
        ,class_acctual
        ,SUM(predict_automatch) AS predict_automatch_sum
        ,SUM(class_acctual) AS class_acctual_sum
    FROM df_sub
    WHERE
        brand_id = '166'
        --AND month_year = '2020-11'
    GROUP BY
         brand_id
        ,prod_gr_id
        --month_year
        --,country_id_n
        ,predict_automatch
        ,class_acctual
    ORDER BY
        prod_gr_id
        ,brand_id ASC
        
    ;
    """
)

In [33]:
prod_gr_id_x_brand_id_CM

,prod_gr_id,brand_id,brand_id_count,predict_automatch,class_acctual,predict_automatch_sum,class_acctual_sum
0,427,166,15,0.0,0.0,0.0,0.0
1,427,166,79,0.0,1.0,0.0,79.0
2,427,166,29,1.0,0.0,29.0,0.0
3,427,166,53,1.0,1.0,53.0,53.0


#### Example of brand_id used by all groups

In [34]:
df2 = prod_gr_id_x_brand_id.pivot(index='brand_id', columns='prod_gr_id', values='brand_id_count').reset_index()
df2.dropna()

prod_gr_id,brand_id,413,426,427
39,184,15.0,15.0,76.0
58,203,28.0,174.0,20.0
87,243,21.0,123.0,15.0
120,290,21.0,76.0,17.0


In [35]:
brand_184 = pysqldf(
        """
        SELECT DISTINCT
             prod_gr_id
            ,brand_id
            ,COUNT(brand_id) as brand_id_count
            --,country_id_n
            ,predict_automatch
            ,class_acctual
            ,SUM(predict_automatch) AS predict_automatch_sum
            ,SUM(class_acctual) AS class_acctual_sum
        FROM df_sub
        WHERE
            brand_id = '184'
            --AND month_year = '2020-11'
        GROUP BY
             brand_id
            ,prod_gr_id
            --month_year
            --,country_id_n
            ,predict_automatch
            ,class_acctual
        ORDER BY
            prod_gr_id
            ,brand_id ASC

        ;
        """)

In [36]:
brand_184

,prod_gr_id,brand_id,brand_id_count,predict_automatch,class_acctual,predict_automatch_sum,class_acctual_sum
0,413,184,2,0.0,0.0,0.0,0.0
1,413,184,3,0.0,1.0,0.0,3.0
2,413,184,2,1.0,0.0,2.0,0.0
3,413,184,8,1.0,1.0,8.0,8.0
4,426,184,4,NaN,0.0,NaN,0.0
5,426,184,11,NaN,1.0,NaN,11.0
6,427,184,16,0.0,0.0,0.0,0.0
7,427,184,2,0.0,1.0,0.0,2.0
8,427,184,4,1.0,0.0,4.0,0.0
9,427,184,54,1.0,1.0,54.0,54.0


In [37]:
brand_203 = pysqldf(
        """
        SELECT DISTINCT
             prod_gr_id
            ,brand_id
            ,COUNT(brand_id) as brand_id_count
            --,country_id_n
            ,predict_automatch
            ,class_acctual
            ,SUM(predict_automatch) AS predict_automatch_sum
            ,SUM(class_acctual) AS class_acctual_sum
        FROM df_sub
        WHERE
            brand_id = '203'
            --AND month_year = '2020-11'
        GROUP BY
             brand_id
            ,prod_gr_id
            --month_year
            --,country_id_n
            ,predict_automatch
            ,class_acctual
        ORDER BY
            prod_gr_id
            ,brand_id ASC

        ;
        """)

In [38]:
brand_203

,prod_gr_id,brand_id,brand_id_count,predict_automatch,class_acctual,predict_automatch_sum,class_acctual_sum
0,413,203,8,0.0,0.0,0.0,0.0
1,413,203,4,0.0,1.0,0.0,4.0
2,413,203,16,1.0,1.0,16.0,16.0
3,426,203,1,NaN,1.0,NaN,1.0
4,426,203,47,0.0,0.0,0.0,0.0
5,426,203,25,0.0,1.0,0.0,25.0
6,426,203,3,1.0,0.0,3.0,0.0
7,426,203,98,1.0,1.0,98.0,98.0
8,427,203,4,0.0,0.0,0.0,0.0
9,427,203,2,0.0,1.0,0.0,2.0


In [39]:
brand_243 = pysqldf(
        """
        SELECT DISTINCT
             prod_gr_id
            ,brand_id
            ,COUNT(brand_id) as brand_id_count
            --,country_id_n
            ,predict_automatch
            ,class_acctual
            ,SUM(predict_automatch) AS predict_automatch_sum
            ,SUM(class_acctual) AS class_acctual_sum
        FROM df_sub
        WHERE
            brand_id = '243'
            --AND month_year = '2020-11'
        GROUP BY
             brand_id
            ,prod_gr_id
            --month_year
            --,country_id_n
            ,predict_automatch
            ,class_acctual
        ORDER BY
            prod_gr_id
            ,brand_id ASC

        ;
        """)

In [40]:
brand_243

,prod_gr_id,brand_id,brand_id_count,predict_automatch,class_acctual,predict_automatch_sum,class_acctual_sum
0,413,243,5,0.0,0.0,0.0,0.0
1,413,243,3,0.0,1.0,0.0,3.0
2,413,243,1,1.0,0.0,1.0,0.0
3,413,243,12,1.0,1.0,12.0,12.0
4,426,243,24,0.0,0.0,0.0,0.0
5,426,243,19,0.0,1.0,0.0,19.0
6,426,243,6,1.0,0.0,6.0,0.0
7,426,243,74,1.0,1.0,74.0,74.0
8,427,243,3,0.0,0.0,0.0,0.0
9,427,243,12,1.0,1.0,12.0,12.0
